In [10]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
from urllib.request import urlopen

# GeoJSON stadsdelen Amsterdam (optioneel)
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
with urlopen(GEOJSON_URL) as response:
    stadsdelen = json.load(response)

# Get Sensors from data (locaties)
df = pd.read_csv("sensor-location.xlsx - Sheet1.csv")

# Splits Lat/Long, strip spaties en zet naar float
df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
df["Lat"]  = pd.to_numeric(df["Lat"].astype(str).str.strip(),  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"].astype(str).str.strip(), errors="coerce")

# Some sensors don't have "Breedte", so it gives "Effectieve breedte"
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

# ---- Crowd sensor data (ALLE timestamps) ----
cs = pd.read_csv("predicted_sensor_values_3min.csv")
cs["timestamp"] = pd.to_datetime(cs["timestamp"], errors="coerce")

time_cols = [c for c in ["timestamp","hour","minute","day","month","weekday","is_weekend"] if c in cs.columns]
sensor_cols = [c for c in cs.columns if c not in time_cols]

# ✅ Juiste sleutelkolom gebruiken (Objectnummer) + schonen
df["sensor"] = df["Objectummer"].astype(str).str.strip()

# Wide -> Long: één rij per (timestamp, sensor)
cs_long = cs.melt(
    id_vars=time_cols,
    value_vars=sensor_cols,
    var_name="sensor",
    value_name="Persons"
)
cs_long["sensor"] = cs_long["sensor"].astype(str).str.strip()

# Zorg dat Persons numeriek is
cs_long["Persons"] = pd.to_numeric(cs_long["Persons"], errors="coerce")

# Koppel waarden aan Lat/Long
merged = cs_long.merge(df[["sensor","Lat","Long"]], on="sensor", how="inner")
merged = merged.dropna(subset=["Lat","Long"]).copy()

# (optioneel) filter sensoren zonder waarde
merged = merged.dropna(subset=["Persons"])

# Sorteren en animatie-as
merged = merged.sort_values(["timestamp","sensor"])
merged["timestamp_str"] = merged["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")

# ---- Sanity checks in console ----
print("Unieke sensoren (cs):", len(sensor_cols))
print("Unieke sensoren (locs):", df["sensor"].nunique())
print("Na merge rijen:", len(merged))
print("Frames:", merged["timestamp_str"].nunique())

# HEATMAP (met animatie over timestamps)
fig = px.density_mapbox(
    merged,
    lat="Lat",
    lon="Long",
    z="Persons",                     # gewicht: meer persons = hogere intensiteit
    radius=60,                       # straal in meters
    center={"lat": 52.372, "lon": 4.900},
    zoom=10,
    color_continuous_scale="YlOrRd", # geel→rood
    opacity=0.7,
    height=700,
    animation_frame="timestamp_str"  # meerdere frames over tijd
)

fig.update_layout(
    mapbox_style="carto-positron",
    margin={"r":0,"t":0,"l":0,"b":0}
)

# ±1 seconde per frame
if fig.layout.updatemenus and fig.layout.updatemenus[0].buttons:
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"] = {"duration": 1000, "redraw": False}
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"] = {"duration": 0}

fig.show()


Unieke sensoren (cs): 48
Unieke sensoren (locs): 36
Na merge rijen: 0
Frames: 0


C:\Users\matsj\AppData\Local\Temp\ipykernel_20392\2872265588.py:65: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [3]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
from urllib.request import urlopen

#GeoJSON stadsdelen Amsterdam
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
with urlopen(GEOJSON_URL) as response:
    stadsdelen = json.load(response)

#Get Sensors from data
df = pd.read_csv("sensor-location.xlsx - Sheet1.csv")
df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
df["Lat"]  = pd.to_numeric(df["Lat"],  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"], errors="coerce")

#Some sensors don't have "Breedte", so it gives "Effectieve breedte"
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

#Crowd sensor data used 
first_timestamp = cs["timestamp"].iloc[0]
cs_first = cs[cs["timestamp"] == first_timestamp]
time_cols = ["timestamp", "hour", "minute", "day", "month", "weekday", "is_weekend"]
sensor_cols = [c for c in cs.columns if c not in time_cols]
cs_first_long = cs_first[sensor_cols].melt(var_name="sensor", value_name="value").sort_values("sensor")
print(cs_first_long.head())

df["Persons"] = cs_first_long["value"]
#

#HEATMAP
fig = px.density_mapbox(
    df,
    lat="Lat",
    lon="Long",
    z="Persons",                     # gewicht: meer persons = hogere intensiteit
    radius=60,                       # straal in meters (speel hiermee!)
    center={"lat": 52.372, "lon": 4.900},
    zoom=10,
    color_continuous_scale="YlOrRd", # geel→rood
    opacity=0.7,
    height=700
)

fig.update_layout(
    mapbox_style="carto-positron",   # of "open-street-map" als je geen token hebt
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()


                     sensor     value
24    actual_CMSA-GAKH-01_0  8.258706
25  actual_CMSA-GAKH-01_180  6.766169
26  actual_CMSA-GAWW-11_120  4.803922
27  actual_CMSA-GAWW-11_300  1.960784
28  actual_CMSA-GAWW-12_115  4.102564


C:\Users\matsj\AppData\Local\Temp\ipykernel_20392\2209383005.py:35: DeprecationWarning: *density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.density_mapbox(
